# Exercise 3

<p style="font-size:15px">In this exercise, the task is to create an application to examine the format of films submitted to The Narbonne Online Film Festival and converting the files in the right format if necessary. For this, we will be using ffprobe to examine the files, and then ffmpeg to convert the files.</p>

<p style="font-size:15px">Let's first import Video from IPython.display. Video will be used to display the files.</p>

In [1]:
pip install termcolor

  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4830 sha256=b4f5f8b250f5ca1a09f550df2dc2393e770be712f09b6f228577d9982df1cd99
  Stored in directory: /home/jovyan/.cache/pip/wheels/3f/e3/ec/8a8336ff196023622fbcb36de0c5a5c218cbb24111d1d4c7f2
Successfully built termcolor
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from IPython.display import Video
from termcolor import colored

<p style="font-size:15px">Next, we will download the latest FFMpeg static build.</p>

In [3]:
exist = !which ffmpeg
if not exist:
  !curl https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz -o ffmpeg.tar.xz \
     && tar -xf ffmpeg.tar.xz && rm ffmpeg.tar.xz
  ffmdir = !find . -iname ffmpeg-*-static
  path = %env PATH
  path = path + ':' + ffmdir[0]
  %env PATH $path

!which ffmpeg

/usr/bin/ffmpeg


<p style="font-size:15px">Here we are just inserting the path of the files into a list, so that we can loop through it later.</p>

In [4]:
filenames = ["Exercise3_Films/Cosmos_War_of_the_Planets.mp4", 
             "Exercise3_Films/Last_man_on_earth_1964.mov", 
             "Exercise3_Films/The_Gun_and_the_Pulpit.avi", 
             "Exercise3_Films/The_Hill_Gang_Rides_Again.mp4", 
             "Exercise3_Films/Voyage_to_the_Planet_of_Prehistoric_Women.mp4"]

<p style="font-size:15px">Now let's take a look at the videos.</p>

In [5]:
Video(filenames[0])

In [6]:
Video(filenames[1])

In [7]:
Video(filenames[2])

In [8]:
Video(filenames[3])

In [9]:
Video(filenames[4])

<p style="font-size:15px">Looking at the displays, videos one and four look to be okay, while the rest definitely seem to have something wrong with them. However, we cannot determine whether a file has passed the requirements set by The Narbonne Online Festival (TNOF) unless we examine them with ffprobe. Here are the requirements provided by TNOF:</p>
<table>
    <tr>
        <th style="text-align : center;">Entries</th>
        <th style="text-align : center;">Format</th>
    </tr>
    <tr>
        <td style="text-align : left;">Video format (container)</td>
        <td style="text-align : center;">mp4</td>
    </tr>
    <tr>
        <td style="text-align : left;">Video codec</td>
        <td style="text-align : center;">h.264</td>
    </tr>
    <tr>
        <td style="text-align : left;">Audio codec</td>
        <td style="text-align : center;">aac</td>
    </tr>
    <tr>
        <td style="text-align : left;">Frame rate</td>
        <td style="text-align : center;">25 FPS</td>
    </tr>
    <tr>
        <td style="text-align : left;">Aspect Ratio</td>
        <td style="text-align : center;">16:9</td>
    </tr>
    <tr>
        <td style="text-align : left;">Resolution</td>
        <td style="text-align : center;">640 x 360</td>
    </tr>
    <tr>
        <td style="text-align : left;">Video bit rate</td>
        <td style="text-align : center;">2 – 5 Mb/s</td>
    </tr>
    <tr>
        <td style="text-align : left;">Audio bit rate</td>
        <td style="text-align : center;">up to 256 kb/s</td>
    </tr>
    <tr>
        <td style="text-align : left;">Audio channels</td>
        <td style="text-align : center;">stereo</td>
    </tr>
</table>

<p style="font-size:15px">
    Let's write the function to examine the files.<br/><br/>
    The function <strong>probe</strong> takes in the list of filepaths. Before looping through the list, a new empty list is initialized. This new list will be where we store the new filepaths of the converted files.<br/><br/>
    For each file, ffprobe is used to extract video and audio streams. From the video stream, codec_name, display_aspect_ratio and bit_rate is retrieved. These values are equivalent to video codec, aspect ratio and video bit rate [1]. From the audio stream, codec_name, bit_rate and channel_layout is retrieved. These values are equivalent to audio codec, audio bit rate and audio channels [2].<br/><br/>
    To get the frames per second of each video, we run a code that calculates the fps [3].<br/><br/>
    The resolution of each file is determined by extracting the width and height from the video stream and converted into the right format (width:height) [4].<br/><br/>
    To simplify the way format is extracted, the function simply splits the file name and gets the extension value [5].<br/><br/>
    Next, the function compares each value to the requirements specified by TNOF. If any requirement has not been met, the function will print the error as well as write it into a report in TXT format. Unless the film has passed all checks, the videos will then be converted into the right format and outputted into a file with "_formatOK" at the end of it's name – indicating it has been formatted. The new file path will be inserted into the previously initialized list, and the list is returned after all files have been checked.<br/>
</p>
<p style="font-size:14px; color:red;">The function will contain comments to explain the code more in-depth.</p>

In [10]:
def probe(files):
    newFiles = []
    
    # loop through file names
    for file in files:
        print("Checking", file, "...")
        # passed is a global boolean that will determine whether a film has passed or not
        passed = True
        
        # [1] get codec_name, display_aspect_ratio and bit_rate from video stream
        video_stream = !ffprobe -v error -select_streams v:0 -show_entries stream=codec_name,display_aspect_ratio,bit_rate -of default=noprint_wrappers=1:nokey=1 $file
        # [2] get codec_name, bit_rate and channel_layout from audio stream
        audio_stream = !ffprobe -v error -select_streams a:0 -show_entries stream=codec_name,bit_rate,channel_layout -of default=noprint_wrappers=1:nokey=1 $file
        # [3] get fps
        fps = !ffmpeg -i $file 2>&1 | sed -n "s/.*, \(.*\) tbr.*/\1/p"
        # [4] get height and width from video stream and convert to the right format for resolution
        resolution = !ffprobe -v error -select_streams v:0 -show_entries stream=width,height -of csv=s=x:p=0 $file
        
        # get format name from the video stream
#         videoformat = !ffprobe -v error -show_entries format=format_name -of default=noprint_wrappers=1:nokey=1 $file
        # [5] alternatively, get video format by getting the extension of the file in index 1
        videoformat = file.split('.')
        
        # the film file name is checked to contain "formatOK" to prevent an empty txt report file from generating
        if "formatOK" not in videoformat[0]:
            # videoformat[0] will be used to open a txt file
            newFile = videoformat[0] + ".txt"
            writeFile = open(newFile, "w")
            
        # check if the film meets the requirement for video format
        if videoformat[1] != "mp4":
            # change global val passed to false
            passed = False
            # generate error statement, print, and write to report
            problem = "Format Error! " + videoformat[1] + " format unsupported. Only mp4 allowed."
            print(colored(problem, 'red'))
            writeFile.write(problem + "\n")
            
        # check if the film meets the requirement for video codec
        if video_stream[0] != "h264":
            # change global val passed to false
            passed = False
            # generate error statement, print, and write to report
            problem = "Video Codec Error! " + video_stream[0] + " video codec unsupported. Only h264 allowed."
            print(colored(problem, 'red'))
            writeFile.write(problem + "\n")
        
        # check if the film meets the requirement for audio codec
        if audio_stream[0] != "aac":
            # change global val passed to false
            passed = False
            # generate error statement, print, and write to report
            problem = "Audio Codec Error! " + audio_stream[0] + " audio codec unsupported. Only aac allowed."
            print(colored(problem, 'red'))
            writeFile.write(problem + "\n")
        
        # check if the film meets the requirement for frames per second
        if int(float(fps[0])) != 25:
            # change global val passed to false
            passed = False
            # generate error statement, print, and write to report
            problem = "FPS Error! " + str(int(float(fps[0]))) + " fps unsupported. Only 25 fps allowed."
            print(colored(problem, 'red'))
            writeFile.write(problem + "\n")
        
        # check if the film meets the requirement for aspect ratio
        if video_stream[1] != "16:9":
            # change global val passed to false
            passed = False
            # generate error statement, print, and write to report
            problem = "Aspect Ratio Error! " + video_stream[1] + " aspect ratio unsupported. Only 16:9 aspect ratio allowed."
            print(colored(problem, 'red'))
            writeFile.write(problem + "\n")
        
        # check if the film meets the requirement for resolution
        if resolution[0] != "640x360":
            # change global val passed to false
            passed = False
            # generate error statement, print, and write to report
            problem = "Resolution Error! " + resolution[0] + " resolution unsupported. Only 640x360 allowed."
            print(colored(problem, 'red'))
            writeFile.write(problem + "\n")
        # check if the film meets the requirement for video bit rate
        if round(float(video_stream[2])/1000000, 2) < 2.0 or round(float(video_stream[2])/1000000, 2) > 5:
            # change global val passed to false
            passed = False
            # generate error statement, print, and write to report
            problem = "Video Bit Rate Error! " + str(round(float(video_stream[2])/1000000, 2)) + " mb/s video bitrate unsupported. Only bitrate between 2 and 5 mb/s allowed."
            print(colored(problem, 'red'))
            writeFile.write(problem + "\n")
        
        # check if the film meets the requirement for audio bit rate
        if round(float(audio_stream[2])/1000, 2) > 256.0:
            # change global val passed to false
            passed = False
            # generate error statement, print, and write to report
            problem = "Audio Bit Rate Error! " + str(round(float(audio_stream[2])/1000, 2)) + " kb/s audio bitrate unsupported. Only bitrate up to 256 kb/s allowed."
            print(colored(problem, 'red'))
            writeFile.write(problem + "\n")
        
        # check if the film meets the requirement for audio channels
        if audio_stream[1] != "stereo":
            # change global val passed to false
            passed = False
            # generate error statement, print, and write to report
            problem = "Audio Channel Error! " + audio_stream[1] + " audio channel unsupported. Only stereo allowed."
            print(colored(problem, 'red'))
            writeFile.write(problem + "\n")
        
        # if all requirements met, film is ok and can be used
        if passed:
            print("Film has passed all checks.\n")
        # if any requirement has not passed, we will convert the format
        else:
            formattedFile = videoformat[0] + "_formatOK.mp4"
            # the code below converts the file into the correct formats
            !ffmpeg -v error -y -i $file -c:av copy -b:v 4M -b:a 256k -r 25 -s 640x360 -aspect 16:9 -vcodec libx264 -acodec aac -ac 2 $formattedFile
            # -v error        : prints errors should there be any
            # -y              : "yes" to all 
            # -i $file        : file to be converted
            # -c:av copy      : copies the video stream from $file into $formattedFile
            # -b:v 4m         : changes the video bit rate to 4 mb/s
            # -b:a 256k       : changes the audio bit rate to 256 kb/s
            # -r 25           : changes the frames per second to 25
            # -s 640x360      : changes the resolution to 640x360
            # -aspect 16:9    : changes the aspect ratio to 16:9
            # -vcodec libx264 : changes the video codec to h264
            # -acodec aac     : changes the audio codec to aac
            # -ac 2           : changes the audio channel to stereo
            # $formattedFile  : changes the video to *.mp4 format
            
            print("Checks and conversions complete for " + file + "\n")    
            # append formattedFile into newFiles
            newFiles.append(formattedFile)
            
    # return newFiles
    return newFiles

<p style="font-size:15px;">Let's invoke the <strong>probe</strong> function and pass in the list of filepaths.</p>

In [11]:
formatted_files = probe(filenames)
formatted_files

Checking Exercise3_Films/Cosmos_War_of_the_Planets.mp4 ...
FPS Error! 29 fps unsupported. Only 25 fps allowed.
Aspect Ratio Error! 314:177 aspect ratio unsupported. Only 16:9 aspect ratio allowed.
Resolution Error! 628x354 resolution unsupported. Only 640x360 allowed.
Audio Bit Rate Error! 317.1 kb/s audio bitrate unsupported. Only bitrate up to 256 kb/s allowed.
Checks and conversions complete for Exercise3_Films/Cosmos_War_of_the_Planets.mp4

Checking Exercise3_Films/Last_man_on_earth_1964.mov ...
Format Error! mov format unsupported. Only mp4 allowed.
Video Codec Error! prores video codec unsupported. Only h264 allowed.
Audio Codec Error! pcm_s16le audio codec unsupported. Only aac allowed.
FPS Error! 23 fps unsupported. Only 25 fps allowed.
Video Bit Rate Error! 9.29 mb/s video bitrate unsupported. Only bitrate between 2 and 5 mb/s allowed.
Audio Bit Rate Error! 1536.0 kb/s audio bitrate unsupported. Only bitrate up to 256 kb/s allowed.
Checks and conversions complete for Exercise3

['Exercise3_Films/Cosmos_War_of_the_Planets_formatOK.mp4',
 'Exercise3_Films/Last_man_on_earth_1964_formatOK.mp4',
 'Exercise3_Films/The_Gun_and_the_Pulpit_formatOK.mp4',
 'Exercise3_Films/The_Hill_Gang_Rides_Again_formatOK.mp4',
 'Exercise3_Films/Voyage_to_the_Planet_of_Prehistoric_Women_formatOK.mp4']

<p style="font-size:15px;">As you can see, each file has problematic fields that do not meet the requirements specified, as well as conversions for each file being done. To check if the newly formatted files are indeed in the right format, we'll pass the new list into the function again.</p>

In [12]:
probe(formatted_files)

Checking Exercise3_Films/Cosmos_War_of_the_Planets_formatOK.mp4 ...
Film has passed all checks.

Checking Exercise3_Films/Last_man_on_earth_1964_formatOK.mp4 ...
Film has passed all checks.

Checking Exercise3_Films/The_Gun_and_the_Pulpit_formatOK.mp4 ...
Film has passed all checks.

Checking Exercise3_Films/The_Hill_Gang_Rides_Again_formatOK.mp4 ...
Film has passed all checks.

Checking Exercise3_Films/Voyage_to_the_Planet_of_Prehistoric_Women_formatOK.mp4 ...
Film has passed all checks.



[]

<p style="font-size:15px;">Now all the files have passed the checks. Let's display the videos as well.</p>

In [13]:
Video(formatted_files[0])

In [14]:
Video(formatted_files[1])

In [15]:
Video(formatted_files[2])

In [16]:
Video(formatted_files[3])

In [17]:
Video(formatted_files[4])

<p style="font-size:15px;">Now we can see that all the films are playing fine, and with the right formats.</p>

## References
<ul>
    <strong>Retreiving with ffprobe</strong>
    <li>
        https://stackoverflow.com/questions/47905083/how-to-check-number-of-channels-in-my-audio-wav-file-using-ffmpeg-command
    </li>
    <strong>Retreiving FPS</strong>
    <li>
        https://askubuntu.com/questions/110264/how-to-find-frames-per-second-of-any-video-file
    </li>
    <strong>Retreiving resolution</strong>
    <li>
        https://superuser.com/questions/841235/how-do-i-use-ffmpeg-to-get-the-video-resolution
    </li>
<!--     <li>
        https://stackoverflow.com/questions/62795409/how-to-find-out-exact-container-format-of-a-video-file-using-ffprobe
    </li> -->
<!--     <li>
        https://stackoverflow.com/questions/2869281/how-to-determine-video-codec-of-a-file-with-ffmpeg
    </li> -->
<!--     <li>
        https://write.corbpie.com/getting-video-bitrate-with-ffprobe/
    </li> -->
    <br/>
    <strong>Coursera Lab</strong>
    <li>
        https://www.coursera.org/learn/uol-cm3065-intelligent-signal-processing/ungradedLab/GoXFL/10-007-exercise-19-processing-video-with-ffmpeg/lab?path=%2Fnotebooks%2FExercises%2FExercise%252019.%2520Processing%2520video%2520with%2520ffmpeg.ipynb
    </li>
    <strong>Converting with ffmpeg</strong>
    <li>
        https://opensource.com/article/17/6/ffmpeg-convert-media-file-formats
    </li>
    <strong>Converting video codec</strong>
    <li>
        https://stackoverflow.com/questions/5678695/ffmpeg-usage-to-encode-a-video-to-h264-codec-format
    </li>
    <strong>Converting audio channel</strong>
    <li>
        https://trac.ffmpeg.org/wiki/AudioChannelManipulation
    </li>
    <strong>Converting aspect ratio</strong>
    <li>
        https://www.youtube.com/watch?v=9EY19Dea534
    </li>
<!--     <li>
        https://trac.ffmpeg.org/wiki/ChangingFrameRate
    </li> -->
<!--     <li>
        https://stackoverflow.com/questions/24087249/ffmpeg-change-resolution-of-the-video-with-aspect-ratio
    </li> -->
<!--     <li>
        https://stackoverflow.com/questions/45464788/resize-and-change-bitrate-with-ffmpeg
    </li> -->
</ul>